# Test: redistribute_for_bounds

This notebook validates the weight redistribution with per-weight lower/upper bounds by projecting onto the box constraints while preserving the total sum via a 1-D Lagrange multiplier root-finding approach.

In [ ]:
import numpy as np
from poly_data.market_selection import redistribute_for_bounds, PositionSizeResult

# Helper to build position_sizes dict from numeric list

def make_positions(vals):
    return {
        f"id_{i}": PositionSizeResult(trade_size=float(v), max_size=float(v))
        for i, v in enumerate(vals)
    }

def arr(d):

    return np.array([d[k] for k in sorted(d.keys())])



In [2]:
# Basic test: simple redistribution within bounds, preserve sum
w = np.array([1.0, 2.0, 3.0, 4.0])
pos = make_positions(w)

floors = {f"id_{i}": 0.5 for i in range(len(w))}
ceilings = {f"id_{i}": 5.0 for i in range(len(w))}

adj = redistribute_for_bounds(pos, floors, ceilings)
adj_vals = np.array([adj[k] for k in sorted(adj.keys())])

print("Original sum:", w.sum())
print("Adjusted sum:", adj_vals.sum())
print("Within bounds:", np.all(adj_vals >= 0.5 - 1e-12) and np.all(adj_vals <= 5.0 + 1e-12))
print("Adjusted:", adj_vals)



Original sum: 10.0
Adjusted sum: 9.999999999999432
Within bounds: True
Adjusted: [1. 2. 3. 4.]


In [5]:
# Test with tight bounds forcing clipping
w = np.array([500.0, 400.0, 30.0, 12.0])
pos = make_positions(w)

floors = {"id_0": 100.0, "id_1": 50.0, "id_2": 100.0, "id_3": 20.0}
ceilings = {"id_0": 1000.0, "id_1": 1000.0, "id_2": 1000.0, "id_3": 1000.0}

adj = redistribute_for_bounds(pos, floors, ceilings)
adj_vals = np.array([adj[k] for k in sorted(adj.keys())])

print("Original sum:", w.sum())
print("Adjusted sum:", adj_vals.sum())
print("Values:", adj_vals)



Original sum: 942.0
Adjusted sum: 942.0000000000002
Values: [461. 361. 100.  20.]


In [4]:
# Infeasible case: sum of floors exceeds target sum
w = np.array([1.0, 1.0, 1.0])
pos = make_positions(w)

floors = {"id_0": 2.0, "id_1": 2.0, "id_2": 2.0}
ceilings = {"id_0": 4.0, "id_1": 4.0, "id_2": 4.0}

try:
    adj = redistribute_for_bounds(pos, floors, ceilings)
    print("Unexpected success, values:", adj)
except Exception as e:
    print("Correctly caught infeasible case:", e)



Correctly caught infeasible case: Infeasible target sum 3; must be in [6, 12] given the bounds.
